In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers, regularizers

### Loading the Data

In [2]:
cvd_data = pd.read_csv(r"Datasets\Analyzed\Analyzed_CVD_Prediction.csv")
cvd_data.head()

,heart_rate,oxygen_saturation,respiratory_rate,temperature,Label
0,93.0,98.0,39.0,36.0,1
1,97.0,98.0,16.0,36.5,0
2,82.0,97.0,25.0,36.3,1
3,98.0,96.0,16.0,36.5,1
4,98.0,99.0,28.0,37.2,1


# Training a DL Model using Keras with 2 Layers

In [3]:
# Split the data into training and validation sets
train_data = cvd_data.sample(frac=0.8, random_state=0)
val_data = cvd_data.drop(train_data.index)

# Preprocess the data
train_labels = train_data.pop('Label')
val_labels = val_data.pop('Label')

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=50,
                    validation_data=(val_data, val_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(val_data, val_labels)

print('Test accuracy:', test_acc)


Epoch 1/50
346/346 [==============================] - 7s 5ms/step - loss: 0.2640 - accuracy: 0.8556 - val_loss: 0.1874 - val_accuracy: 0.8962
Epoch 2/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1973 - accuracy: 0.8934 - val_loss: 0.1936 - val_accuracy: 0.9222
Epoch 3/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1832 - accuracy: 0.9023 - val_loss: 0.1711 - val_accuracy: 0.9016
Epoch 4/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1939 - accuracy: 0.8983 - val_loss: 0.1763 - val_accuracy: 0.9237
Epoch 5/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1826 - accuracy: 0.8984 - val_loss: 0.1677 - val_accuracy: 0.9164
Epoch 6/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1834 - accuracy: 0.8988 - val_loss: 0.1639 - val_accuracy: 0.8947
Epoch 7/50
346/346 [==============================] - 1s 4ms/step - loss: 0.1783 - accuracy: 0.9012 - val_loss: 0.1685 - val_accuracy: 0.8925
Epoch 

#### These scores seems good to be deploy, so it will be converted into a TFLite model

### Converting this model to TFLite Model

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  # <-- ensure Pi compatibility
tflite_model = converter.convert()

with open('Models/CVD_Classification_Model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\SANDIN~1\AppData\Local\Temp\tmpipxnl7o0\assets
